In [ ]:
# Przykład ustawienia środowiska z określoną datą i strefą czasową
from rocketpy import Environment

env = Environment(
    date=(2025, 12, 18, 12),  # rok, miesiąc, dzień, godzina
    timezone="Europe/Warsaw"
)

In [ ]:
env.set_location(latitude=35.35, longitude=-117.81)

In [ ]:
env.set_elevation("Open-Elevation")

In [ ]:
env.plots.atmospheric_model()

In [ ]:
from math import exp
from rocketpy import Fluid, LiquidMotor, CylindricalTank, MassFlowRateBasedTank, Rocket, Flight

In [ ]:
# 1. DEFINICJA PŁYNÓW (WARUNKI: 60 BAR, ~27.5°C)
# N2O w 60 barach jest blisko punktu krytycznego - jest znacznie rzadszy niż woda!
# Gęstość cieczy spada do ok. 710 kg/m3.
# Gęstość gazu (pary nasyconej) jest bardzo wysoka - ok. 185 kg/m3.
oxidizer_liq = Fluid(name="N2O_l", density=710)
oxidizer_gas = Fluid(name="N2O_g", density=185)

# Etanol jest cieczą mało ściśliwą, spadek gęstości wynika głównie z temperatury (27.5C).
fuel_liq = Fluid(name="ethanol_l", density=782)
# Gaz w zbiorniku paliwa (jeśli są pęcherzyki) to opary etanolu,
# ale w zbiorniku tłokowym ich masa jest pomijalna.
fuel_gas = Fluid(name="ethanol_g", density=1.6)

# 2. GEOMETRIA (Dopasowana do Twoich wymiarów: r=9.3cm)
# Musisz stworzyć dwie osobne geometrie, bo mają różne długości (90cm i 50cm).
ox_tank_shape = CylindricalTank(radius = 0.093, height = 1.2, spherical_caps = True)
fuel_tank_shape = CylindricalTank(radius = 0.093, height = 0.5, spherical_caps = True)

# 3. ZBIORNIKI (Masy wyliczone dla wsadu 25.5 kg)

oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=ox_tank_shape,
    flux_time=11.2,
    initial_liquid_mass=19.2,       # Wyliczona masa N2O
    initial_gas_mass=0.1,           # Niewielka ilość pary nad cieczą
    liquid_mass_flow_rate_in=0,
    # Zaktualizowałem licznik w lambda, aby startował od realnej masy
    liquid_mass_flow_rate_out=lambda t: 1.7142857,
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)

fuel_tank = MassFlowRateBasedTank(
    name="fuel tank",
    geometry=fuel_tank_shape,
    flux_time=11.2,
    initial_liquid_mass=6.3,        # Wyliczona masa Etanolu
    initial_gas_mass=0.001,         # W tłokowym zbiorniku paliwa gazu powinno być 0
    liquid_mass_flow_rate_in=0,
    # Zaktualizowałem licznik w lambda
    liquid_mass_flow_rate_out=0.5624,
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,       # Gaz nie wypływa z paliwa w układzie tłokowym
    liquid=fuel_liq,
    gas=fuel_gas,
)

In [ ]:
# Draw the tank
oxidizer_tank.draw()
fuel_tank.draw()


In [ ]:
# Evolution of the Propellant Mass and the Mass flow rate
oxidizer_tank.fluid_mass.plot()
oxidizer_tank.net_mass_flow_rate.plot()

In [ ]:
# Evolution of the Propellant center of mass position
oxidizer_tank.center_of_mass.plot()

In [ ]:
example_liquid = LiquidMotor(
    thrust_source="C:\\Users\\krikb\\Desktop\\simulations\\data\\AGH-SS_Z4000-10sBurn-optimal.eng",
    dry_mass=2.7,
    dry_inertia=(0.0203499, 0.0203499, 0.003375000000000001),
    nozzle_radius=0.075,
    center_of_dry_mass_position=1.75,
    nozzle_position=0,
    burn_time=14.4,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)
example_liquid.add_tank(tank=oxidizer_tank, position=1.0)
example_liquid.add_tank(tank=fuel_tank, position=2.5)

In [ ]:
example_liquid.info()

In [ ]:
example_liquid.all_info()

In [ ]:
TRB = Rocket(
    radius=0.2,
    mass=58.367,
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="C:\\Users\\krikb\\Desktop\\simulations\\data\\powerOffDragCurve.csv",
    power_on_drag="C:\\Users\\krikb\\Desktop\\simulations\\data\\powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

In [ ]:
TRB.add_motor(example_liquid, position=-1)

In [ ]:

rail_buttons = TRB.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.6182,
    angular_position=45,
)

In [ ]:
nose_cone = TRB.add_nose(
    length=0.7, kind="von karman", position=4
)

fin_set = TRB.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.060,
    span=0.110,
    position=-1.04956,
    cant_angle=0.5,
)

tail = TRB.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)

In [ ]:
main = TRB.add_parachute(
    name="main",
    cd_s=10.0,
    trigger=800,      # ejection altitude in meters
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
    radius=1.5,
    height=1.5,
    porosity=0.0432,
)

drogue = TRB.add_parachute(
    name="drogue",
    cd_s=1.0,
    trigger="apogee",  # ejection at apogee
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
    radius=1.5,
    height=1.5,
    porosity=0.0432,
)

In [ ]:
TRB.plots.static_margin()

In [ ]:
test_flight = Flight(
    rocket=TRB, environment=env, rail_length=5.2, inclination=85, heading=0
    )

In [ ]:
TRB.draw()